In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import requests
from PIL import Image, ImageOps
import cv2
from scipy import cluster
from sklearn.preprocessing import MinMaxScaler
import imagesize
import random
from sklearn.preprocessing import LabelEncoder
from ipywidgets import interact
import time
from io import BytesIO
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.callbacks import EarlyStopping
from keras import Sequential
from keras import layers, models
from keras.preprocessing.image import ImageDataGenerator
from tqdm.notebook import tqdm
from keras.metrics import Precision, Recall, Accuracy
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from math import ceil
import os
from preproc.attributes import AttributePreproc


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#!pip install --upgrade pip

In [ ]:
#!git clone -b preproc_package https://github.com/orlandoalexander/Moda-Personal-Assistant.git

In [ ]:
#%cd Moda-Personal-Assistant
#! pip install .

In [ ]:
#path = '/content/Moda-Personal-Assistant/img'

In [ ]:
#prep = Preproc(path, (299,299), 'fabric', 0.2)

In [ ]:
#X_train, X_test, y_train, y_test = prep.run()

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
#!unzip '/content/drive/MyDrive/Le Wagon/Moda-Personal-Assistant/Data/DeepFashion_categorical_images_low/img.zip'

In [ ]:
#PATH_IMG = '/content/img/'
#PATH_DATA = '/content/drive/MyDrive/Le Wagon/Moda-Personal-Assistant/Data/DeepFashion_attributes - train'


In [ ]:
PATH_IMG = 'Data/DeepFashion_attributes_image/'
PATH_DATA = 'Data/DeepFashion_attributes_data'


# Attributes

## Load raw data

In [ ]:
cat_names = pd.read_csv(PATH_DATA+'/list_category.txt',sep ='\s+',header=None).reset_index().rename(columns={0:'category',1:'section','index':'cat_num'})


In [ ]:
cat_names['cat_num'] = cat_names['cat_num'].apply(lambda x : x+1)

In [ ]:
cat_names.head()

In [ ]:
cat_tag = pd.read_csv(PATH_DATA+'/tag_cat_simple.txt',sep='\s+',header=None,names=['category']).rename(columns={'category':'cat_num'})



In [ ]:
cat_img = pd.read_csv(PATH_DATA+'/img_simple.txt',sep='\s+',header=None,names=['category']).rename(columns={'category':'img'})

In [ ]:
cat_num = cat_img.join(cat_tag).drop(columns='img')


In [ ]:
cat_num.head()

In [ ]:
cat = cat_num.merge(cat_names, on='cat_num', how='left').drop(columns='cat_num')



In [ ]:
cat

In [ ]:
cat['section'] = cat['section'].map({1:'upper',2:'lower',3:'full body'})

In [ ]:
cat_names = pd.read_csv(PATH_DATA+'/list_category.txt',sep ='\s+',header=None).reset_index().rename(columns={0:'category',1:'section','index':'cat_num'})



In [ ]:
cat_names['cat_num'] = cat_names['cat_num'].apply(lambda x : x+1)

In [ ]:
cat_names.head()

In [ ]:
cat_tag = pd.read_csv(PATH_DATA+'/tag_cat_simple.txt',sep='\s+',header=None,names=['category']).rename(columns={'category':'cat_num'})



In [ ]:
cat_img = pd.read_csv(PATH_DATA+'/img_simple.txt',sep='\s+',header=None,names=['category']).rename(columns={'category':'img'})

In [ ]:
cat_num = cat_img.join(cat_tag)


In [ ]:
cat_num.head()

In [ ]:
cat = cat_num.merge(cat_names, on='cat_num', how='left').drop(columns='cat_num')


In [ ]:
cat['img'] = cat['img'].apply(lambda x: x[4:])

In [ ]:
cat

In [ ]:
cat['section'] = cat['section'].map({1:'upper',2:'lower',3:'full body'})

In [ ]:
attr_names = pd.read_csv(PATH_DATA+'/list_attr_simple.txt',sep='\s+', header=None).rename(columns={0:'attribute',1:'attribute_type'})


In [ ]:
attr_names

In [ ]:
attr_names['attribute_type'] = attr_names['attribute_type'].map({1:'design',2:'sleeves',3:'length',4:'part',5:'fabric',6:'fit'})


In [ ]:
attr_names[attr_names['attribute_type']=='sleeves'].iloc[0,0]

In [ ]:
np.where(attr_names['attribute_type'].values=='sleeves')[0]

In [ ]:
attr_names_headers = attr_names.iloc[:,0]
attr_names_headers

In [ ]:
attr_tags = pd.read_csv(PATH_DATA+'/tags_attr_simple.txt',sep='\s+',header=None,names=attr_names_headers)


In [ ]:
attr_tags

In [ ]:
attr_img = pd.read_csv(PATH_DATA+'/img_simple.txt',sep='\s+',header=None).rename(columns={0:'img'})



In [ ]:
attr_img

In [ ]:
attr = attr_img.join(attr_tags,how='inner').drop(columns='img')

In [ ]:
attr

In [ ]:
bb = pd.read_csv(PATH_DATA+'/bbox.txt',sep='\s+',header=None,index_col=False, names=['img','x_1', 'y_1', 'x_2', 'y_2'])


In [ ]:
bb['img'] = bb['img'].apply(lambda x: x[4:])

In [ ]:
landmarks = pd.read_csv(PATH_DATA+'/landmarks.txt',sep='\s+',names=['img','clothes_type','v1','x1','y1','v2','x2','y2','v3','x3','y3','v4','x4','y4','v5','x5','y5','v6','x6','y6','v7','x7','y7','v8','x8','y8']).fillna(0).drop(columns=['clothes_type'])



In [ ]:
landmarks['img'] = landmarks['img'].apply(lambda x: x[4:])

In [ ]:
data_full = cat.join(attr,how='left').merge(bb,how='left',on='img').merge(landmarks, how='left',on='img')



In [ ]:
data_full = cat

In [ ]:
data_full

In [ ]:
cat_counts = data_full.category.value_counts() # number of images in each section class (list of tuples) in training data set


In [ ]:
cat_counts.mean()

In [ ]:
cat_counts

## Filter attributes

In [ ]:
data_full

In [ ]:
# If item is sleevless and of lower type, set attribute value to 0:
data_full['sleeveless'] = data_full.apply(lambda x: 0 if ((x['sleeveless'])==1) & (x['section']=='lower') else x['sleeveless'], axis =1)


In [ ]:
data_full

In [ ]:
data_full.drop(columns=['no_dress','no_neckline'],inplace=True)

## Crop and augment images

In [ ]:
data_full

In [ ]:
data_full_attr = data_full.iloc[:,3:-28]

In [ ]:
data_full_attr

In [ ]:
attribute_count = data_full_attr.apply(pd.value_counts)

In [ ]:
attribute_count_T = attribute_count.T.reset_index().rename(columns={'index':'attribute'})

In [ ]:
attribute_count_T

In [ ]:
attribute_count_type = attribute_count_T.merge(attr_names, on='attribute')

In [ ]:
attribute_count_type

In [ ]:
attribute_count_type_mean = attribute_count_type.groupby('attribute_type').median().drop(columns=0)

In [ ]:
attribute_count_type_mean

In [ ]:
attribute_count_mean = attribute_count_type[1].median()
attribute_count_mean

In [ ]:
attr_names_list = attribute_count_type['attribute'].values
attr_names_list

In [ ]:
def landmark_crop(index): # NOT USED AT THE MOMENT !
    observation = data_full.iloc[index,:] # get observation from database according to index
    v1,x1,y1,v2,x2,y2,v3,x3,y3,v4,x4,y4=observation[-24:-12] # landmarks belonging to ALL images
    lm_x = [x1,x2,x3,x4]
    lm_y = [y1,y2,y3,y4]
    lm_v = [v1,v2,v3,v4]
    
    section = data_full.iloc[index,2] # get section type for image
    if section == 'upper' or section == 'full body': # if image is an upper or full body, has 6 landmarks
        v5,x5,y5,v6,x6,y6=observation[-12:-6]
        lm_x.extend([x5,x6])
        lm_y.extend([y5,y6])
        lm_v.extend([v5,v6])
    if section == 'full body': # if image is full body has 8 landmarks
        v7,x7,y7,v8,x8,y8=observation[-6:]
        lm_x.extend([x7,x8])
        lm_y.extend([y7,y8])
        lm_v.extend([v7,v8])
    
    lm_x = np.array(lm_x)
    lm_y = np.array(lm_y)
    lm_x = lm_x[lm_x !=0] # keep only non-zero landmarks 
    lm_y = lm_y[lm_y !=0]
    
    pts = (np.array(list(zip(lm_x,lm_y)))).astype('int') # store coordinates defined by landmarks
    
    img_name = observation[0]
    full_path = PATH_IMG+img_name
    
    img = cv2.imread(full_path) # open image using OpenCV
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # convert image to correct format
    
    x,y,w,h = cv2.boundingRect(pts) # create approximate rectangle using landmark points
    cropped = Image.fromarray(img[y:y+h, x:x+w].copy()) # crop image by bounding rectangle

    return cropped
    

In [ ]:
def format_image(index): # format image
    observation = data_full.iloc[index,:] # get observation from database according to index    
    img_name = observation[0]
    full_path = PATH_IMG+img_name

    # LOAD IMAGES:
    img = Image.open(full_path)
  
    # CROP IMAGES:
    box_x1,box_y1,box_x2,box_y2 = (np.array(observation[-28:-24].values)) # box coords
    cropped = img.crop((box_x1, box_y1, box_x2, box_y2))

    # PAD IMAGES:
    cropped_pad = ImageOps.pad(cropped,(256,256),color=(255, 255, 255)) # pad image to (256,256) with white background
    return cropped_pad


In [ ]:
sample_index = np.random.randint(20000)
image = format_image(sample_index)


plt.imshow(image)

In [ ]:
# fig, axs = plt.subplots(24,10,figsize=(30,100))
# for i in range(24):
#     for j in range(10):
#         index = data_full[data_full[attr_names_list[i]]==1].sample(1).index._data[0]
#         img = format_image(index)
#         img = axs[i][j].imshow(img)
#         axs[i][j].title.set_text(attr_names_list[i])

## Model testing 

In [ ]:
def plot_loss_accuracy(history, title=None):
    fig, ax = plt.subplots(1,2, figsize=(20,7))
    
    # --- LOSS --- 
    
    ax[0].plot(history.history['loss'])
    ax[0].plot(history.history['val_loss'])
    ax[0].set_title('Model loss')
    ax[0].set_ylabel('Loss')
    ax[0].set_xlabel('Epoch')
    ax[0].set_ylim((0,3))
    ax[0].legend(['Train', 'Test'], loc='best')
    ax[0].grid(axis="x",linewidth=0.5)
    ax[0].grid(axis="y",linewidth=0.5)
    
    # --- ACCURACY
    
    ax[1].plot(history.history['accuracy'])
    ax[1].plot(history.history['val_accuracy'])
    ax[1].set_title('Model Accuracy')
    ax[1].set_ylabel('Accuracy')
    ax[1].set_xlabel('Epoch')
    ax[1].legend(['Train', 'Test'], loc='best')
    ax[1].set_ylim((0,1))
    ax[1].grid(axis="x",linewidth=0.5)
    ax[1].grid(axis="y",linewidth=0.5)
    
    if title:
        fig.suptitle(title)

In [ ]:
def format_image_model(img_name,box_x1,box_y1,box_x2,box_y2): # format image
    full_path = PATH_IMG+img_name

    
    # LOAD IMAGES:
    img = cv2.imread(full_path) # open image using OpenCV
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # CROP IMAGES:
    cropped = Image.fromarray(img[box_y1:box_y2, box_x1:box_x2]) # crop image using box
    
    # PAD IMAGES:
    cropped_pad = ImageOps.pad(cropped,(256,256),color=(255,255,255)) # pad image to (256,256) with white background

    cropped_pad_array = np.asarray(cropped_pad)
    
    return cropped_pad_array

### Without image augmentation

In [ ]:
def format_image_model(img_name,box_x1,box_y1,box_x2,box_y2): # format image
    full_path = PATH_IMG+img_name

    
    # LOAD IMAGES:
    img = cv2.imread(full_path) # open image using OpenCV
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # CROP IMAGES:
    cropped = Image.fromarray(img[box_y1:box_y2, box_x1:box_x2]) # crop image using box
    
    # PAD IMAGES:
    cropped_pad = ImageOps.pad(cropped,(256,256),color=(255,255,255)) # pad image to (256,256) with white background

    cropped_pad_array = np.asarray(cropped_pad)
    
    return cropped_pad_array

##### VGG16

In [ ]:
# Training image size --> (256,256,3)

In [ ]:
model_vgg16 = VGG16(weights="imagenet", include_top=False, input_shape=fabric_X_train[0].shape) # include top removes classification layer from imagenet dataset
model_vgg16.trainable = False # not trainable weights

## Preprocessing input (normalise data for VGG16 model):
fabric_X_train = preprocess_input(fabric_X_train) 
fabric_X_val = preprocess_input(fabric_X_val)
fabric_X_test = preprocess_input(fabric_X_test)


In [ ]:
flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(60, activation='relu')
dense_layer_2 = layers.Dense(30, activation='relu')
prediction_layer = layers.Dense(6, activation='softmax')


model = models.Sequential([
    model_vgg16,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['recall','precision','accuracy'],
)


es = EarlyStopping(monitor='val_recall', patience=20,  restore_best_weights=True)

history = model.fit(fabric_X_train, fabric_y_train, epochs=50, validation_data=fabric_X_val, batch_size=32, callbacks=[es])

#### Augmentation testing

In [ ]:
fabric_data = data_full.iloc[:,np.r_[0,18:24,27:31]] # select all attributes related to fabric

In [ ]:
fabric_X_names = fabric_data.iloc[:,np.r_[0,7,8,9,10]]

In [ ]:
fabric_X_train = np.array([format_image_model(fabric_X_name[0], *fabric_X_name[1:]) for fabric_X_name in tqdm(fabric_X_names.values)])




In [ ]:
fabric_X_train.shape

In [ ]:
fabric_y_train = fabric_data.iloc[:,1:7]

In [ ]:
fabric_y_train.shape

In [ ]:
fabric_X_train, fabric_X_test, fabric_y_train, fabric_y_test = train_test_split(fabric_X_train, fabric_y_train, test_size =0.2,random_state=2)


In [ ]:
fabric_X_train.shape

In [ ]:
fabric_X_train, fabric_X_val, fabric_y_train, fabric_y_val = train_test_split(fabric_X_train, fabric_y_train, test_size =0.2,random_state=2)


#### Baseline model

In [ ]:
def initialize_model():
  model = Sequential()
  model.add(layers.Conv2D(16, (4,4), input_shape=fabric_X_train[0].shape, padding='valid', activation="relu"))
  model.add(layers.MaxPooling2D(pool_size = (2,2)))
  model.add(layers.Conv2D(32, (3,3), padding='valid', activation="relu"))
  model.add(layers.MaxPooling2D(pool_size = (2,2)))
  model.add(layers.Conv2D(64, (2,2), input_shape=fabric_X_train[0].shape, padding='valid', activation="relu"))
  model.add(layers.MaxPooling2D(pool_size = (2,2)))
  model.add(layers.Flatten())
  model.add(layers.Dense(60,activation='relu'))
  model.add(layers.Dense(6, activation = 'softmax'))
  model.summary()
  return model

In [ ]:
model = initialize_model()
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=[Recall(), Precision(), Accuracy()],
)

In [ ]:
es = EarlyStopping(patience=20,  restore_best_weights=True)

history = model.fit(fabric_X_train, np.array(fabric_y_train), epochs=50, validation_data=(fabric_X_val,fabric_y_val), batch_size=16, callbacks=[es])

In [ ]:
model.evaluate(fabric_X_test,fabric_y_test)

In [ ]:
plot_loss_accuracy(history)


#### VGG16

In [ ]:
# Training image size --> (256,256,3)

In [ ]:
model_vgg16 = VGG16(weights="imagenet", include_top=False, input_shape=fabric_X_train[0].shape) # include top removes classification layer from imagenet dataset
model_vgg16.trainable = False # not trainable weights

## Preprocessing input (normalise data for VGG16 model):
fabric_X_train = preprocess_input(fabric_X_train) 
fabric_X_val = preprocess_input(fabric_X_val)
fabric_X_test = preprocess_input(fabric_X_test)


In [ ]:
flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(60, activation='relu')
dense_layer_2 = layers.Dense(30, activation='relu')
prediction_layer = layers.Dense(6, activation='softmax')


model = models.Sequential([
    model_vgg16,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['recall','precision','accuracy'],
)


es = EarlyStopping(monitor='val_recall', patience=20,  restore_best_weights=True)

history = model.fit(fabric_X_train, fabric_y_train, epochs=50, validation_data=fabric_X_val, batch_size=32, callbacks=[es])

### With image augmentation

In [ ]:
fabric_data_augment = data_full.iloc[:,np.r_[0,18:24,27:31]] # select all attributes related to fabric
fabric_data_augment

In [ ]:
fabric_mean = round(fabric_data_augment.iloc[:,1:7].apply(pd.value_counts).iloc[1,:].mean())
fabric_mean

In [ ]:
fabric_attrs = list(fabric_data_augment.iloc[:,1:7].apply(pd.value_counts).iloc[1,:].items())
fabric_attrs

In [ ]:
def format_image_model(img_name,box_x1,box_y1,box_x2,box_y2): # format image
    full_path = PATH_IMG+img_name
    
    # LOAD IMAGES:
    img = Image.open(full_path)
  
    # CROP IMAGES:
    cropped = img.crop((box_x1, box_y1, box_x2, box_y2))
    
    # PAD IMAGES:
    cropped_pad = ImageOps.pad(cropped,(256,256),color=(255,255,255)) # pad image to (256,256) with white background

    cropped_pad_array = np.asarray(cropped_pad)
    
    return cropped_pad_array

In [ ]:
fabric_data_augment_array = pd.DataFrame()
for  attr, count in fabric_attrs:

  if count >= fabric_mean: # if number of samples is greater than mean number of samples for 'attr'
    print(attr)
    sample_df = fabric_data_augment[fabric_data_augment[attr]==1].sample(fabric_mean,random_state=2)
    sample_df['img'] = sample_df.apply(lambda row: format_image_model(row[0], *row[7:]),axis=1)
    sample_df = sample_df.iloc[:,:7]
  else:
    print(attr)
    sample_values = [0]*count
    remaining_sum = fabric_mean-count
    i = 0
    while remaining_sum != 0:
      sample_values[i]+=1
      remaining_sum-=1
      i = (i+1)%len(sample_values)
    sample_df = pd.DataFrame()
    for index, row in enumerate(fabric_data_augment[fabric_data_augment[attr]==1].values):
      sample = sample_values[index]
      cropped_img_array = format_image_model(row[0], *row[7:])
      temp_oversample_df = pd.concat([pd.DataFrame([row], columns = fabric_data_augment.columns).iloc[:,:7]]*(sample+1),ignore_index=True)
      temp_oversample_df.iloc[0,0] = [cropped_img_array]
      if sample != 0:
        augmented_img_array = augment(cropped_img_array,sample)
        temp_oversample_df.iloc[1:,0] = augmented_img_array
      sample_df = pd.concat([sample_df,temp_oversample_df],axis=0)
  fabric_data_augment_array = pd.concat([fabric_data_augment_array,sample_df],axis=0)

In [ ]:
fabric_data_augment_array

In [ ]:
img_t = fabric_data_augment_array.iloc[37,0]

In [ ]:
plt.imshow(img_t)

In [ ]:
def augment(img_array, samples):
  img_array = img_array.reshape((1,) + img_array.shape)

  # AUGMENT IMAGES:

  # Create an ImageDataGenerator object with the desired transformations
  datagen = ImageDataGenerator(
      horizontal_flip=True,
      width_shift_range=0.2, 
      rotation_range=15,
      fill_mode='constant',
      cval=255
  )

  aug_iter = datagen.flow(img_array, batch_size=1)
  arrays = [aug_iter.next()[0].astype('uint8') for i in range (samples)]
  return arrays

In [ ]:
def display_augmented():
  sample_index = np.random.randint(20000)
  img = format_image(sample_index)
  img_array = np.asarray(img)
  img_array = img_array.reshape((1,) + img_array.shape)

  # AUGMENT IMAGES:

  # Create an ImageDataGenerator object with the desired transformations
  datagen = ImageDataGenerator(
      horizontal_flip=True,
      width_shift_range=0.2, 
      rotation_range=15,
      fill_mode='constant',
      cval=255
  )

  aug_iter = datagen.flow(img_array, batch_size=1)

  # generate samples and plot
  fig, ax = plt.subplots(nrows=1, ncols=5, figsize=(15,15))

  # generate batch of images
  for i in range(5):

    # convert to unsigned integers
    image = next(aug_iter)[0].astype('uint8')
  
    # plot image
    ax[i].imshow(image)
    ax[i].axis('off')

In [ ]:
display_augmented()

In [ ]:
fabric_y_train.apply(pd.value_counts).iloc[1,:]

In [ ]:
def format_image_model(img_name,box_x1,box_y1,box_x2,box_y2): # format image
    full_path = PATH_IMG+img_name

    
    # LOAD IMAGES:
    img = cv2.imread(full_path) # open image using OpenCV
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # CROP IMAGES:
    cropped = Image.fromarray(img[box_y1:box_y2, box_x1:box_x2]) # crop image using box
    
    # PAD IMAGES:
    cropped_pad = ImageOps.pad(cropped,(256,256),color=(255,255,255)) # pad image to (256,256) with white background

    cropped_pad_array = np.asarray(cropped_pad)
    
    return cropped_pad_array

#### Fabric

In [ ]:
fabric_data = data_full.iloc[:,np.r_[0,18:24,27:31]] # select all attributes related to fabric

In [ ]:
fabric_X_names = fabric_data.iloc[:,np.r_[0,7,8,9,10]]

In [ ]:
fabric_X_train = np.array([format_image_model(fabric_X_name[0], *fabric_X_name[1:]) for fabric_X_name in tqdm(fabric_X_names.values)])



In [ ]:
fabric_X_train.shape

In [ ]:
fabric_y_train = fabric_data.iloc[:,1:7]

In [ ]:
fabric_y_train.shape

In [ ]:
fabric_X_train, fabric_X_test, fabric_y_train, fabric_y_test = train_test_split(fabric_X_train, fabric_y_train, test_size =0.2,random_state=2)


In [ ]:
fabric_X_train.shape

In [ ]:
fabric_X_train, fabric_X_val, fabric_y_train, fabric_y_val = train_test_split(fabric_X_train, fabric_y_train, test_size =0.2,random_state=2)


##### Baseline model

In [ ]:
def initialize_model():
  model = Sequential()
  model.add(layers.Conv2D(16, (4,4), input_shape=fabric_X_train[0].shape, padding='valid', activation="relu"))
  model.add(layers.MaxPooling2D(pool_size = (2,2)))
  model.add(layers.Conv2D(32, (3,3), padding='valid', activation="relu"))
  model.add(layers.MaxPooling2D(pool_size = (2,2)))
  model.add(layers.Conv2D(64, (2,2), input_shape=fabric_X_train[0].shape, padding='valid', activation="relu"))
  model.add(layers.MaxPooling2D(pool_size = (2,2)))
  model.add(layers.Flatten())
  model.add(layers.Dense(60,activation='relu'))
  model.add(layers.Dense(6, activation = 'softmax'))
  model.summary()
  return model

In [ ]:
model = initialize_model()
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=[Recall(), Precision(), Accuracy()],
)

In [ ]:
es = EarlyStopping(patience=20,  restore_best_weights=True)

history = model.fit(fabric_X_train, np.array(fabric_y_train), epochs=50, validation_data=(fabric_X_val,fabric_y_val), batch_size=16, callbacks=[es])

In [ ]:
model.evaluate(fabric_X_test,fabric_y_test)

In [ ]:
plot_loss_accuracy(history)


##### VGG16

In [ ]:
# Training image size --> (256,256,3)

In [ ]:
model_vgg16 = VGG16(weights="imagenet", include_top=False, input_shape=fabric_X_train[0].shape) # include top removes classification layer from imagenet dataset
model_vgg16.trainable = False # not trainable weights

## Preprocessing input (normalise data for VGG16 model):
fabric_X_train = preprocess_input(fabric_X_train) 
fabric_X_val = preprocess_input(fabric_X_val)
fabric_X_test = preprocess_input(fabric_X_test)


In [ ]:
flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(60, activation='relu')
dense_layer_2 = layers.Dense(30, activation='relu')
prediction_layer = layers.Dense(6, activation='softmax')


model = models.Sequential([
    model_vgg16,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['recall','precision','accuracy'],
)


es = EarlyStopping(monitor='val_recall', patience=20,  restore_best_weights=True)

history = model.fit(fabric_X_train, fabric_y_train, epochs=50, validation_data=fabric_X_val, batch_size=32, callbacks=[es])

### Nigel model

In [ ]:
from sys import getsizeof

In [ ]:
img = Image.open(PATH_IMG+'Crocheted_Gauze_Maxi_Dress/img_00000003.jpg')

In [ ]:
img_array_normal = np.asarray(img)

In [ ]:
normal = getsizeof(img_array_normal)/1073741824
normal

In [ ]:
img_array = np.asarray(img)/255

In [ ]:
standardised = getsizeof(img_array)/1073741824
standardised

In [ ]:
plt.imshow(img_array)

In [ ]:
# from preproc.attributes import AttributePreproc
# PATH_IMG = 'Data/DeepFashion_attributes_image/'
# prep = AttributePreproc(PATH_IMG, 
#                         (256,256), 
#                         'sleeves',
#                         0.2)
# X_train, X_test, y_train, y_test  = prep.run()

In [ ]:
prep = AttributePreproc(PATH_IMG, 
                        (256,256), 
                        'sleeves',
                        0.2) 

In [ ]:
X_train, X_test, y_train, y_test  = prep.run()

In [ ]:
for i in dir(prep):
    del i
del prep

In [ ]:
preprocess = Lambda(mobilenet.preprocess_input, input_shape=(256,256,3))


In [ ]:
Input(X_train.shape)

In [ ]:

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

from keras.layers import Input, Dense, Flatten, GlobalAveragePooling2D, Lambda
from keras import Sequential
from keras.metrics import Precision, Recall
from keras.optimizers import Adam

from keras.applications import InceptionV3
from keras.applications import inception_v3
from keras.applications import ResNet50
from keras.applications import resnet
from keras.applications import MobileNetV2
from keras.applications import mobilenet_v2
from keras.applications import EfficientNetB0
from keras.applications import efficientnet
from keras.applications import mobilenet


# attribute should be one of the following strings:
# 'design', 'sleeves', 'length', 'neckline', 'fabric', 'fit'

# model should be one of the following strings:
# 'inception', 'resnet', 'mobilenet', 'efficientnet'

class AttrModel:
    def __init__(self, attribute, epochs, model, input_shape, batch_size, final_layer_neurons,
                 X_train, X_test,
                 y_train, y_test, **kwargs):
        self.attribute = attribute
        self.model = model.lower()
        self.input_shape = input_shape
        self.batch_size = batch_size
        self.epochs = epochs
        self.final_layer_neurons = final_layer_neurons
        self.kwargs = kwargs
        self.X_train, self.X_val, self.y_train, self.y_val = train_test_split(
            X_train, y_train, test_size=.5
        )
        self.X_test = X_test
        self.y_test = y_test
        del X_train, y_train, y_test

        self.cat_nums = {
            'design': 7,
            'sleeves': 3,
            'length': 2,
            'neckline': 4,
            'fabric': 6,
            'fit': 3
        }
        self.cat_num = self.cat_nums[self.attribute]
        self.activation = 'sigmoid' if self.attribute == 'length' else 'softmax'
        self.loss = 'binary_crossentropy' if self.attribute == 'length' else 'categorical_crossentropy'
        self.model = self.instantiate_model()  # calling the function below


    def instantiate_model(self):
        # import ipdb;
        # ipdb.set_trace()
        input = Input(self.input_shape)
        preprocess = Lambda(mobilenet.preprocess_input)

        if self.model == 'inception':              # calling the chosen pretrained model
            base_model = InceptionV3(include_top=False, weights='imagenet',
                                    classes=self.cat_num, input_tensor=input)
        elif self.model == 'resnet':
            base_model = ResNet50(include_top=False, weights='imagenet',
                                  classes=self.cat_num, input_tensor=input)
        elif self.model == 'mobilenet':
            base_model = MobileNetV2(include_top=False, weights='imagenet',
                                     classes=self.cat_num, input_tensor=input)
        elif self.model == 'efficientnet':
            base_model = EfficientNetB0(include_top=False, weights='imagenet',
                                        classes=self.cat_num, input_tensor=input)
        else:
            print('''No model found. Please pass one of the following:
                  inception, resnet, mobilenet, efficientnet''')

        base_model.trainable = False    # freeze layers
        
        pool = GlobalAveragePooling2D()
        dense = Dense(units=self.final_layer_neurons, activation='relu')
        prediction = Dense(units=self.cat_num, activation=self.activation)
        model = Sequential([preprocess,base_model, pool, dense, prediction])
        print(self.loss) # check loss func
        model.compile(loss=self.loss, optimizer='adam',
                      metrics=['accuracy', Precision(), Recall()])

        return model

    def train(self):
        # import ipdb
        # ipdb.set_trace()
        self.model.history = self.model.fit(
            self.X_train, self.y_train,
            epochs=self.epochs, batch_size=self.batch_size,
            validation_split=0.2, verbose=1, 
            callbacks=[EarlyStopping(patience=2, restore_best_weights=True)])
        return self.model.history

    def finetune(self,callbacks=[EarlyStopping(patience=3)],
                 loss='categorical_crossentropy',
                 metrics=['accuracy', Precision(), Recall()]):
        self.model.trainable = True     # unfreeze layers, then compile to save changes
        self.model.compile(
            optimizer=Adam(1e-5),                  # Very low learning rate
            loss=loss,
            metrics=metrics)
        self.model.fit(self.X_val, self.y_val,
                       epochs=self.epochs, batch_size=self.batch_size,
                       callbacks=[EarlyStopping(patience=2, monitor='val_accuracy', 
                                                restore_best_weights=True)],
                       validation_split=0.2)
        return self.model.history

    def evaluate(self):
        return self.model.evaluate(self.X_test, self.y_test, verbose=1)

    def predict(self, X):
        return self.model.predict(X)


In [ ]:
inception = AttrModel(attribute='neckline',
                      model='mobilenet',
                      input_shape=(256,256,3),
                      batch_size= 16,
                      epochs=50,
                      final_layer_neurons=100,
                      X_train=X_train,
                      X_test=X_test,
                      y_train=y_train,
                      y_test=y_test)

In [ ]:
inception.train()

# Categories

## Load raw data

In [ ]:
cat_names = pd.read_csv(PATH_DATA+'/list_category.txt',sep ='\s+',header=None).reset_index().rename(columns={0:'category',1:'section','index':'cat_num'})


In [ ]:
cat_names['cat_num'] = cat_names['cat_num'].apply(lambda x : x+1)

In [ ]:
cat_names.head()

In [ ]:
cat_tag = pd.read_csv(PATH_DATA+'/tag_cat_simple.txt',sep='\s+',header=None,names=['category']).rename(columns={'category':'cat_num'})



In [ ]:
cat_img = pd.read_csv(PATH_DATA+'/img_simple.txt',sep='\s+',header=None,names=['category']).rename(columns={'category':'img'})

In [ ]:
cat_num = cat_img.join(cat_tag).drop(columns='img')


In [ ]:
cat_num.head()

In [ ]:
cat = cat_num.merge(cat_names, on='cat_num', how='left').drop(columns='cat_num')



In [ ]:
cat

In [ ]:
cat['section'] = cat['section'].map({1:'upper',2:'lower',3:'full body'})

In [ ]:
cat_names = pd.read_csv(PATH_DATA+'/list_category.txt',sep ='\s+',header=None).reset_index().rename(columns={0:'category',1:'section','index':'cat_num'})



In [ ]:
cat_names['cat_num'] = cat_names['cat_num'].apply(lambda x : x+1)

In [ ]:
cat_names.head()

In [ ]:
cat_tag = pd.read_csv(PATH_DATA+'/tag_cat_simple.txt',sep='\s+',header=None,names=['category']).rename(columns={'category':'cat_num'})



In [ ]:
cat_img = pd.read_csv(PATH_DATA+'/img_simple.txt',sep='\s+',header=None,names=['category']).rename(columns={'category':'img'})

In [ ]:
cat_num = cat_img.join(cat_tag)


In [ ]:
cat_num.head()

In [ ]:
cat = cat_num.merge(cat_names, on='cat_num', how='left').drop(columns='cat_num')


In [ ]:
cat['img'] = cat['img'].apply(lambda x: x[4:])

In [ ]:
cat

In [ ]:
cat['section'] = cat['section'].map({1:'upper',2:'lower',3:'full body'})

In [ ]:
cat

In [ ]:
cat_counts = cat.category.value_counts() # number of images in each section class (list of tuples) in training data set


In [ ]:
cat.category.nunique()

In [ ]:
cat_counts.mean()

In [ ]:
cat

In [ ]:
cat_counts

In [ ]:
cat_names_list = cat_counts.index

In [ ]:
cat_names_list

In [ ]:
fig, axs = plt.subplots(45,10,figsize=(30,100))
for i in range(45):
    for j in range(10):
        img_name = cat[cat.category==cat_names_list[i]].sample(1, replace=True).img.values[0]
        img = Image.open(PATH_IMG+img_name)
        img = axs[i][j].imshow(img)
        axs[i][j].title.set_text(cat_names_list[i])

In [ ]:
section = pd.read_csv('Data/DeepFashion_sections_data'+'/img.txt',sep=' ',names = ['img'],header=None,index_col=False).fillna(0)
section = section[~section['img'].str.contains('_back')]

# Fabric:
fabric = pd.read_csv('Data/DeepFashion_sections_data'+'/fabric.txt',sep=' ',names = ['img','upper_fabric','lower_fabric'],header=None,index_col=False).fillna(0)
section = section.merge(fabric,on='img').drop(columns='upper_fabric')

# Landmarks:
landmarks = pd.read_csv('Data/DeepFashion_sections_data'+'/keypoints_loc.txt',sep=' ',usecols=[0,15,16,17,18,29,30,31,32,39,40,41,42],names=['img','x1','y1','x2','y2','x3','y3','x4','y4','x5','y5','x6','y6'], header=None,index_col=False)

# Create full data set:
data_full = section.merge(landmarks, on='img',how='left').fillna(0)

# Drop invalid rows:
data_full = data_full[~data_full.isin([-1]).any(1)] # drop rows with inavlid values



In [ ]:
data_full

In [ ]:
replace_cats = dict()

In [ ]:
with open ('Data/DeepFashion_sections_data/joggers.txt','r') as f:
    replace_cats['Joggers'] = f.readlines()[0].split(',')

In [ ]:
with open ('Data/DeepFashion_sections_data/baggy.txt','r') as f:
    replace_cats['Baggy'] = f.readlines()[0].split(',')

In [ ]:
with open ('Data/DeepFashion_sections_data/suiting.txt','r') as f:
    replace_cats['Suiting'] = f.readlines()[0].split(',')

In [ ]:
rename_cats = {'Blouses_Shirts':'Blouses', 'Cardigans':'Blouses',
       'Rompers_Jumpsuits':'Rompers', 'Shirts_Polos':'Shirts', 'Sweatshirts_Hoodies':'Sweaters', 'Tees_Tanks':'Tees',
              'Jackets_Coats':'Jackets', 'Jackets_Vests':'Jackets'}



In [ ]:
dir_path = 'Data/Other/DeepFashion_multimodal_images'
num_files = len(os.listdir(dir_path))
img_cats = np.zeros((num_files,2),dtype=object)
for index,path in enumerate(os.listdir(dir_path)):
    # check if current path is a file
    if path != '.DS_Store' and os.path.isfile(os.path.join(dir_path, path)):
        img_cat = path.split('-')[1]
        if img_cat=='Sweaters' and path.split('-')[0]=='MEN':
            img_cat = 'Shirts'
        replace_cat = [key for key, value in replace_cats.items() if path in value] 
        if replace_cat:
            img_cat = replace_cat[0]
        rename_cat = [value for key, value in rename_cats.items() if img_cat in key]
        if rename_cat:
            img_cat = rename_cat[0]
        img_cats[index,:]=np.array([path,img_cat])    

In [ ]:
img_cats.shape

In [ ]:
img_cats_df = pd.DataFrame(img_cats,columns=['img','category'])

In [ ]:
img_cats_df

In [ ]:
data_full = data_full.merge(img_cats_df, on='img', how='left')

In [ ]:
data_full

In [ ]:
data_full['category'] = np.where(((data_full['category']=='Denim') | (data_full['category']=='Pants')) & (data_full['lower_fabric']==0),'Jeans',  data_full['category'])



In [ ]:
data_full.category = np.where((data_full['category']=='Denim'),'Pants',data_full['category'])

In [ ]:
data_full

In [ ]:
cat_counts = data_full.category.value_counts()

In [ ]:
cat_names_list = cat_counts.index
cat_names_list

In [ ]:
PATH_IMG = 'Data/Other/DeepFashion_multimodal_images/'

In [ ]:
fig, axs = plt.subplots(15,10,figsize=(30,100))
for i in range(15):
    for j in range(10):
        img_name = data_full[data_full.category==cat_names_list[i]].sample(1, replace=True).img.values[0]
        img = Image.open(PATH_IMG+img_name)
        img = axs[i][j].imshow(img)
        axs[i][j].title.set_text(cat_names_list[i])

## Filter categories

In [ ]:
data_full

In [ ]:
# If item is sleevless and of lower type, set attribute value to 0:
data_full['sleeveless'] = data_full.apply(lambda x: 0 if ((x['sleeveless'])==1) & (x['section']=='lower') else x['sleeveless'], axis =1)


In [ ]:
data_full

In [ ]:
data_full.drop(columns=['no_dress','no_neckline'],inplace=True)

# Sections

In [ ]:
section = pd.read_csv('Data/DeepFashion_sections_data'+'/fabric.txt',sep=' ',names = ['img','upper_original','lower_original','outer','upper','lower'],header=None,index_col=False).fillna(0)
section['back'] = np.where(section['img'].str.contains('_back'),1,0)
section['full body'] = np.where((section['img'].str.contains('WOMEN-Dresses|WOMEN-Rompers')) & ~section['img'].str.contains('_back'),1,0)
section['outfit'] = np.where((((section['upper_original']!=7) |(section['lower_original']!=7)) & ((section['upper_original']!=7) |(section['outer']!=7))) & (section['full body']==0) & (section['back']==0),1,0)
section = section.drop(columns=['outer','upper_original','lower_original'])

# Landmarks:
landmarks = pd.read_csv('Data/DeepFashion_sections_data'+'/keypoints_loc.txt',sep=' ',usecols=[0,15,16,17,18,29,30,31,32,39,40,41,42],names=['img','x1','y1','x2','y2','x3','y3','x4','y4','x5','y5','x6','y6'], header=None,index_col=False)

# Create full data set:
data_full = section.merge(landmarks, on='img',how='left').dropna()

# Create upper and lower oberservations (to be cropped later):
upper = data_full[data_full['outfit']==1]
lower = data_full[data_full['outfit']==1]
upper['upper'] = upper['upper'].apply(lambda x: 1)
upper['outfit'] = upper['outfit'].apply(lambda x: 0)
lower['lower'] = lower['lower'].apply(lambda x: 1)
lower['outfit'] = lower['outfit'].apply(lambda x: 0)
data_full = pd.concat([data_full,upper,lower],axis=0)
data_full = data_full[(data_full['outfit']==1) | (data_full['full body']==1) | (data_full['back']==1) | (data_full['upper']==1) | (data_full['lower']==1)]

# Drop invalid rows:
data_full = data_full[~data_full.isin([-1]).any(1)] # drop rows with inavlid values



In [ ]:
data_full